# Aquarium Plan post-processing

In [1]:
# login to Benchling & Aquarium
from aqbt.tools import config_to_sessions
from aqbt.tools import parse_config
import toml

def config(config_path):
    with open(config_path, "r") as f:
        return parse_config(toml.load(f))


def sessions(config):
    return config_to_sessions(config)


sessions = sessions(config('config.toml'))

benchling = sessions['default']['benchling']
registry = sessions['default']['registry']
session = sessions['default']['aquarium']
sessions

{'default': {'aquarium': <AqSession(name=None, AqHTTP=<AqHTTP(user='vrana', url='http://52.27.43.242')>), parent=94009983669088)>,
  'benchling': <benchlingapi.session.Session at 0x7f2e504d2340>,
  'registry': <aqbt.aquarium.registry.LabDNARegistry at 0x7f2e504ef550>}}

In [2]:
PLAN_ID = 39388
# http://52.27.43.242//plans?plan_id=39388
# session.Plan.find(39380)


## Optimize Plan

In [3]:
from pydent.planner import Planner

with session.with_cache(timeout=120) as sess:
    print("PLAN {}".format(PLAN_ID))
    plan = sess.Plan.find(PLAN_ID)

    print('loading planner...')
    planner = Planner(plan)

    print('optimizing...')
    planner.optimize()
    
    
    
    
    print("splitting plans...")
    plans = planner.split()
    print('\tsplit into {} plans'.format(len(plans)))
    
    for i, p in enumerate(plans):
        p.name = 'Part {}/{}: {}'.format(i+1, len(plans), plan.name)
        print('Saving plan to server. Please be patient.')
        
        print('\tPlan: {}'.format(p.name))
        print('\tNumOps: {}'.format(len(p.operations)))
        print('\tprettifying...')
        p.prettify()
        p.save()
        print('\t' + p.url)

PLAN 39388
loading planner...
optimizing...
splitting plans...
	split into 3 plans
Saving plan to server. Please be patient.
	Plan: Part 1/3: Plant TF__1595361543_(Terrarium v0.1.5)
	NumOps: 123
	prettifying...
	http:/52.27.43.242/plans?plan_id=39389
Saving plan to server. Please be patient.
	Plan: Part 2/3: Plant TF__1595361543_(Terrarium v0.1.5)
	NumOps: 155
	prettifying...
	http:/52.27.43.242/plans?plan_id=39390
Saving plan to server. Please be patient.
	Plan: Part 3/3: Plant TF__1595361543_(Terrarium v0.1.5)
	NumOps: 100
	prettifying...
	http:/52.27.43.242/plans?plan_id=39391


In [12]:
plans = planner.split()
for p in plans:
    print("")

In [15]:
plans[0].url

'http:/52.27.43.242/plans?plan_id=None'

In [7]:
from pydent.planner import Planner

with session.with_cache(timeout=120) as sess:
    for i, p in enumerate(plans):
        print('plan {}'.format(i))
        print('post processing...')
        planner = Planner(sess.Plan.find(p.plan.id))

        sess.browser.get([plan], {"operations": {'operation_type': {}, 'field_values': ['sample', 'item']}})

        for op in planner.operations:
            if op.operation_type.name == 'Order Primer':
                op.input('Urgent?').value == 'yes'
            elif op.operation_type.name == 'Run Gel':
                fv = op.input('Gel')
                pour_gel = planner.create_operation_by_name("Pour Gel")
                try:
                    planner.quick_wire(pour_gel, op)
                    print("success")
                except:
                    pass
            elif op.operation_type.name == 'Order gBlock Fragment':
                fv = op.input('Bases')
                fv.value = op.outputs[0].sample.properties['Sequence']
            elif op.operation_type.name == 'Make Glycerol Stock':
                fv = op.input('Needs Sequencing Results?')
                fv.value = 'Yes'
        print('prettifying...')
        planner.prettify()
        planner.save()
        print('saving...')

plan 0
post processing...
success
prettifying...
saving...
plan 1
post processing...
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
prettifying...
saving...
plan 2
post processing...
success
success
success
success
success
success
prettifying...
saving...
